In [4]:
import pandas as pd
import sqlite3
import requests

In [8]:
# Tente conectar-se ao banco de dados (ele será criado se não existir)
conn = sqlite3.connect("usuarios.db")

# Crie um cursor
cursor = conn.cursor()

# Crie a tabela "usuarios" com as colunas necessárias (se não existir)
cursor.execute("""
CREATE TABLE IF NOT EXISTS usuarios (
    id INTEGER PRIMARY KEY,
    nome TEXT,
    conta TEXT,
    agencia TEXT,
    cartao TEXT
);
""")

# Commit para salvar as alterações e fechar a conexão
conn.commit()
conn.close()


In [19]:
# URL da API
api_url = "https://sdw-2023-prd.up.railway.app/users"

# Ler o arquivo CSV usando o Pandas
df = pd.read_csv("dados.csv")

# Inicializar a conexão com o banco de dados SQLite
conn = sqlite3.connect("usuarios.db")
cursor = conn.cursor()

for index, row in df.iterrows():
    # Crie um objeto de usuário com os dados do DataFrame
    user_data = {
        "name": row["nome"],
        "account": {
            "number": row["conta"],
            "agency": row["agencia"],
            "balance": 0,
            "limit": 0
        },
        "card": {
            "number": row["cartao"],
            "limit": 0
        }
    }

    # Faça a solicitação POST para a API para criar o usuário
    response = requests.post(api_url, json=user_data)

    # Verifique se a solicitação foi bem-sucedida
    if response.status_code == 201:
        # Extrair o ID retornado da API
        user_id = response.json()["id"]
        print(f"Usuário {row['nome']} criado com sucesso! ID: {user_id}")

        # Inserir o usuário no banco de dados SQLite com o ID
        cursor.execute("INSERT INTO usuarios (id, nome, conta, agencia, cartao) VALUES (?, ?, ?, ?, ?)", 
                       (user_id, row["nome"], row["conta"], row["agencia"], row["cartao"]))
        conn.commit()
    else:
        print(f"Falha ao criar o usuário {row['nome']}")

# Fechar a conexão com o banco de dados
conn.close()


Usuário Natalia criado com sucesso! ID: 5782


In [20]:
# Inicializar a conexão com o banco de dados SQLite
conn = sqlite3.connect("usuarios.db")
cursor = conn.cursor()

# Consulta SQL para selecionar todos os registros da tabela "usuarios"
query = "SELECT * FROM usuarios"

# Executar a consulta
cursor.execute(query)

# Recuperar todos os registros em uma lista de dicionários
registros = cursor.fetchall()

# Fechar a conexão com o banco de dados
conn.close()

# Exibir os registros
for registro in registros:
    print("ID:", registro[0])
    print("Nome:", registro[1])
    print("Conta:", registro[2])
    print("Agência:", registro[3])
    print("Cartão:", registro[4])
    print("-------------------------")


ID: 5780
Nome: Leandro
Conta: 7071987
Agência: 7890
Cartão: 7890-1234-5678-9012
-------------------------
ID: 5781
Nome: Rafael
Conta: 5787
Agência: 4567-7
Cartão: ****-5678-9012-3456
-------------------------
ID: 5782
Nome: Natalia
Conta: 5978
Agência: 4267-7
Cartão: ****-****-9012-3456
-------------------------
